In [1]:
import pandas as pd
import numpy as np
import ipywidgets as w
from IPython.display import HTML
import bqplot as bq
import copy
import traitlets as tr

In [125]:
MXF_ICON = w.Image(
    value=open("mxf-open-circle-icon.png", "rb").read(),
    format='png',
    width=60,
    height=40,
)

MAP_COLORS = {'Heating': '#c22324',
 'Domestic Hot Water': '#ff7f0e',
 'Heating & Domestic Hot Water': '#eb4f18',
 'Cooling': '#1f77b4',
 'Auxiliary': '#9467bd',
 'Lighting': '#17becf',
 'Small Power': '#d3d3d3',
 'IT/Servers': '#8c564b',
 'PV Generation': '#bcbd22',
 'Catering': '#2ca02c',
 'Lifts': '#e377c2',
 'Other': '#000000',
 'Unknown': '#7C7270'}

TITLE = "<h1>NHS Archetypes - Decarbonisation Pathways Analysis and Visualisation</h1>"
DESCRIPTION = """<ul>
  <li>This app is a dummy demo, illustrating the type of front-end that can be expected for visualising different decarbonisation pathways</li>
  <li>It runs on <code>python</code> code in the browser and can be deployed as a static HTML website</li>
  <li>This is an illustrative example but the calculation would be developed to consider the different archetypes within the NHS portfolio</li>
  <li>Choose from the possible design interventions to understand the incremental savings that are available (the numbers are made up)</li>
</ul> 
"""

SELECT_DESCRIPTION = "<b>Cntrl+click to Select multiple possible the design interventions:</b>"
Y_AXIS_LABEL = "Building Carbon Content (tonnes of CO2/yr)"
FIGURE_TITLE = "Annual Carbon"

def calc_savings(data, selected):
    data = data.copy()
    keys = list(data.keys())
    for n, x in enumerate(selected):
        data[x] = data[keys[n]] - (data[keys[n]] * savings[x])
        keys = list(data.keys())
    return data

def create_plot(x, y, colors):
    # create two vectors x and y to plot a bar chart
    # 1. Create the scales
    xs = bq.OrdinalScale()  # ordinal scale to represent categorical data
    ys = bq.LinearScale()
    
    # 2. Create the axes for x and y
    xax = bq.Axis(scale=xs, label="", grid_lines="none")  # no grid lines needed for x
    yax = bq.Axis(
        scale=ys, orientation="vertical", label=Y_AXIS_LABEL #, tick_format=".0%"
    )  # note the use of tick_format to format ticks
    
    # 3. Create a Bars mark by passing in the scales
    bar = bq.Bars(x=x, y=y, scales={"x": xs, "y": ys}, padding=0.5, colors=colors)
    
    # 4. Create a Figure object by assembling marks and axes
    fig = bq.Figure(marks=[bar], axes=[xax, yax], title=FIGURE_TITLE)
    return fig, bar

def update_plot(x, y, bar):
    # use the hold_sync() context manager
    with bar.hold_sync():
        bar.x = x
        bar.y = y

def get_xy(data):
    return list(data.keys()), np.transpose(np.array(list(data.values())))

class EnergyConsumption(w.VBox):     
    total_saving = tr.Float()

    @tr.observe("total_saving")
    def obs_total_saving(self, on_change):
        self.html_total_saving.value = f"<b>Total Saving</b> = {self.total_saving}%"
        
    def __init__(self, data: dict[str, np.array], savings, colors, selected = []):
        if "existing building" not in data:
            raise ValueError("`existing building` data missing.")
        self.data = data
        self.savings = savings
        self.colors = colors
        self.html_total_saving = w.HTML()
        self.hbx_title = w.HBox([MXF_ICON, w.HTML(TITLE)])
        self.select = w.SelectMultiple(value=selected, options=list(self.savings.keys()), layout={"height": "150px", "width": "400px"})
        self.vbx_select = w.VBox([w.HTML(SELECT_DESCRIPTION), self.select])
        self.hbx_topbar = w.HBox([w.VBox([self.hbx_title, self.html_total_saving, w.HTML(DESCRIPTION)]), self.vbx_select], layout={"justify_content":"space-between"})
        x, y = get_xy(self.data)
        self.fig, self.bar = create_plot(x, y, self.colors)
        super().__init__([self.hbx_topbar, self.fig])
        self._init_controls()

    def _init_controls(self):
        self.select.observe(self._update)

    def _update(self, on_change):
        self.calc_data = calc_savings(self.data, self.select.value)
        x, y = get_xy(self.calc_data)
        update_plot(x, y, self.bar)
        _ = list(chart.calc_data.values())
        self.total_saving = np.round((_[0].sum() - _[-1].sum()) / _[0].sum(), 2)*100

        
labels = ['Heating', 'Domestic Hot Water'] # , 'Auxiliary', 'Lighting', 'Small Power', 'PV Generation'
colors = [MAP_COLORS[x] for x in labels]
data = {
    "existing building": np.array([60, 20]),
}
savings = {
    "replace glazing": 0.1,
    "improve air tightness": 0.1,
    "add roof insulation": 0.1,
    "replace gas heating for air source heat pump": 0.7
}

chart = EnergyConsumption(data, savings, colors)
display(chart)

EnergyConsumption(children=(HBox(children=(VBox(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x…